In [3]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset,ConcatDataset
import numpy as np
from pytorch_forecasting import MAPE
from torch.autograd import Variable

# Configure Random seeds
torch.manual_seed(777)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(777)


In [4]:
train_data = np.load('train_series.npy')
trainset=[]
train_x = train_data[:,:,0]
train_y = train_data[:,:,1]

print(type(train_x[:5]))

<class 'numpy.ndarray'>


In [5]:
# Define custom dataset - train set
class HW5_trainDataset(Dataset):
    
    def __init__(self):


        train_data = np.load('train_series.npy')
        
        train_x = train_data[:,:,0]
        train_y = train_data[:,:,1]
        
        
        self.x_data=torch.from_numpy(train_x).float()
        self.y_data=torch.from_numpy(train_y).float()

        self.len= len(train_x)
 
    def __getitem__(self,index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

# Define custom dataset - test set    
class HW5_testDataset(Dataset):
    
    def __init__(self):
        
        test_data = np.load('test_series.npy')
        
        test_x = test_data[:,:,0]
        test_y = test_data[:,:,1]
        
        self.len= test_x.shape[0]
        self.x_data=torch.from_numpy(test_x).float()
        self.y_data=torch.from_numpy(test_y).float()
        
        
    def __getitem__(self,index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len


# Hyper parameter setting
batch_size = 32

# Create datsets and dataloader
trainset = HW5_trainDataset()
testset = HW5_testDataset()
train_loader =DataLoader(trainset, batch_size = batch_size, shuffle = True)
test_loader= DataLoader(testset, batch_size = batch_size, shuffle = False)

In [6]:
class LSTM(nn.Module):
    def __init__(self, input_size=5, hidden_size=5, num_layers=1, output_size=5):
        super(LSTM, self).__init__()
        self.num_layers = num_layers 
        self.input_size = input_size 
        self.hidden_size = hidden_size 
        self.output_size = output_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) 
        self.fc1 =  nn.Linear(hidden_size, 128) 
        self.fc2 = nn.Linear(128, output_size) 

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
       
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hn = hn.view(-1, self.hidden_size) 
        out = self.relu(hn)
        out = self.fc1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc2(out) #Final Output
        return out
 

In [7]:
model = LSTM()
loss_function = MAPE()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)


params = list(model.parameters())
print("The number of parameters:", sum([p.numel() for p in model.parameters() if p.requires_grad]), "elements")


The number of parameters: 1653 elements


In [8]:
print(model)

LSTM(
  (lstm): LSTM(5, 5, batch_first=True)
  (fc_1): Linear(in_features=5, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=5, bias=True)
  (relu): ReLU()
)


In [75]:
epochs = 100

# train the model
valid_loss_min = int(1e9)
for epoch in range(epochs):
    
    train_loss=0.0
    valid_loss=0.0
    
    # Train 
    
    model.train()
    # loop for ever batch set in train dataset loader
    for i, (data, target) in enumerate(train_loader):
        data = data.reshape(-1,1,5)
        # initialize optimizer
        optimizer.zero_grad()
        
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_size),
                        torch.zeros(1, 1, model.hidden_size))
        
        # train
        y_pred = model(data)
        
        # caculate batch loss
        loss = loss_function(y_pred, target)
        
        # update
        loss.backward(retain_graph=True)
        optimizer.step()
    
        # Record train loss
        train_loss += loss.item() 
    
    # Validate 
    
    model.eval()
    # loop for ever batch set in test dataset loader
    for i, (data, target) in enumerate(test_loader):
        data = data.reshape(-1,1,5)
        y_pred = model(data)
        # calculate the batch loss
        loss = loss_function(y_pred, target)
        
        # calculate validataion loss
        valid_loss += loss.item()
        
        
    # calculate average losses
    train_loss = train_loss / len(train_loader)
    valid_loss = valid_loss / len(test_loader)
    
    # print training/validation results
    print('Epoch: {}     Training Loss: {}    Validation Loss: {}'.format(epoch,train_loss, valid_loss))
    
    # if trained model perform best vaildation loss, then save it to the checkpoint
    if valid_loss <= valid_loss_min:
        print('Decreased Validation Loss ({} ==> {})   < model saved >'.format(valid_loss_min,valid_loss))
        
        checkpoint = {
            'state_dict': model.state_dict(),
        }
        torch.save(checkpoint, './bestModel.pt')
        valid_loss_min = valid_loss


        

    

Epoch: 0     Training Loss: 1.0290913022756576    Validation Loss: 0.9625849843025207
Decreased Validation Loss (1000000000 ==> 0.9625849843025207)   < model saved >
Epoch: 1     Training Loss: 0.8734748996496201    Validation Loss: 0.7947734022140502
Decreased Validation Loss (0.9625849843025207 ==> 0.7947734022140502)   < model saved >
Epoch: 2     Training Loss: 0.7773243390321731    Validation Loss: 0.692499788403511
Decreased Validation Loss (0.7947734022140502 ==> 0.692499788403511)   < model saved >
Epoch: 3     Training Loss: 0.7297347221970558    Validation Loss: 0.663724578499794
Decreased Validation Loss (0.692499788403511 ==> 0.663724578499794)   < model saved >
Epoch: 4     Training Loss: 0.7049571027755738    Validation Loss: 0.6127844413518906
Decreased Validation Loss (0.663724578499794 ==> 0.6127844413518906)   < model saved >
Epoch: 5     Training Loss: 0.6908256575465203    Validation Loss: 0.575508530497551
Decreased Validation Loss (0.6127844413518906 ==> 0.5755085

In [76]:
# Load best performance model
checkpoint = torch.load('./bestModel.pt')
trained_model = LSTM()

trained_model.load_state_dict(checkpoint['state_dict'])

# evaluate
trained_model.eval()
with torch.no_grad():
    test_data = np.load('test_series.npy')
    # load test dataset
    test_x = test_data[:,:,0]
    test_y = test_data[:,:,1]

    
    test_x=torch.from_numpy(test_x).float()
    test_y=torch.from_numpy(test_y).float()
    test_x = test_x.reshape(-1,1,5)

    y_pred = trained_model(test_x)
    loss = loss_function(y_pred, test_y)
    print("accuracy in terms of the mean absolute percentage error (MAPE):", loss.item())

accuracy in terms of the mean absolute percentage error (MAPE): 0.4647793471813202
